In [1]:
from itertools import combinations
from datetime import timedelta
from collections import defaultdict

import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.stats import pearsonr

from tqdm.auto import tqdm

In [22]:
df = pd.read_csv("generated/Aix-en-Provence/measures_100.csv")

In [23]:
df["timestamp"] = pd.to_datetime(df["string(d)"],
                                 format="%Y-%m-%d %H:%M:%S")
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df["lon"], df["lat"], 
                                crs="EPSG:2154")
                                # crs="EPSG:4326")
)
# Sort rows by timestamp for binary searching
gdf.sort_values(by="timestamp", inplace=True)

In [28]:
# Rendez-vous params
dt = timedelta(minutes=1)  # temporal constraint
ds = 500 # spatial constraint (in meters)
min_num_pairs = 200  # minimum number of close measuremennts to consider a rendezvous

d = defaultdict(lambda: [])

# Group the measurements by cyclists
grouped = gdf.groupby("name_of_agent")
# Iterate over every measurement pairs to check closeness condition
# Outer loop iterates over all possbible sensor pairs
for (sensor1, group1), (sensor2, group2) in \
        tqdm(list(combinations(grouped, 2)), desc="Finding rendez-vous pairs"):
    # Inner loop iterate over the measurement pairs
    for _, row_i in group1.iterrows():
        # Look for measurements that satisfy the temporal constraint first
        # Use binary search to quickly find a starting point to search
        timestamp_key = row_i["timestamp"] - dt
        starting_idx = group2["timestamp"].searchsorted(timestamp_key)
        if starting_idx >= len(group2):
            break
        for j in range(starting_idx, len(group2)):
            row_j = group2.iloc[j]
            # Break early
            if np.abs(row_i.timestamp - row_j.timestamp) > dt:
                # Since the table is sorted by time, 
                # following rows will surely be outside of the time window
                break
            # Now we check the spatial constraint
            if np.abs(row_i.geometry.distance(row_j.geometry)) > ds:
                continue
            # Save the measurement pair when it satisfies both constraints
            d[(sensor1, sensor2)].append((row_i.name, row_j.name))

# Remove the sets that have too few measurement pairs
d = {k: pairs for k, pairs in d.items() if len(pairs) >= min_num_pairs}

Pairs:   0%|          | 0/4950 [00:00<?, ?it/s]

In [51]:
# Add noise to a certain sensor
faulty_sensor_name = "worker[1266]"
gdf_noisy = gdf.copy()
for pollutant_type in ["NO2", "O3", "PM10", "PM25"]:
    col = gdf_noisy.loc[gdf_noisy["name_of_agent"] == faulty_sensor_name, f"conc_map'{pollutant_type}'"]
    col += 4 * np.random.randn(len(col))

In [52]:
# Compute correlation coeffs
corr_dict = defaultdict(lambda: [])
pollutant_keys = ["NO2", "O3", "PM10", "PM25"]
for sensor_pair in d.keys():
    x_ids, y_ids = tuple(zip(*d[sensor_pair]))
    x_ids, y_ids = list(x_ids), list(y_ids)

    for p_key in pollutant_keys:
        x = gdf_noisy.loc[list(x_ids)][f"conc_map'{p_key}'"]
        y = gdf_noisy.loc[list(y_ids)][f"conc_map'{p_key}'"]
        
        corr = np.abs(pearsonr(x, y)[0])
        corr_dict[sensor_pair].append(corr)
        
corr_df = pd.DataFrame.from_dict(corr_dict, orient='index', columns=pollutant_keys)

In [59]:
# Rendezvous pairs consisting of the faulty sensor are highlighted in red
# TODO: not sure how to interpret these results, 
#       since the correlation is still there for the faulty sensor...
corr_df.abs().style.apply(lambda x: ['background: darkred; color: white' 
                                  if (faulty_sensor_name in x.name)
                                  else '' for i in x], axis=1)

,NO2,O3,PM10,PM25
"('leisure[16]', 'leisure[236]')",0.538642,0.743873,0.536856,0.547480
"('leisure[173]', 'worker[666]')",0.404024,0.522650,0.174577,0.510620
"('leisure[220]', 'leisure[349]')",0.013490,0.364552,0.156854,0.017341
"('leisure[220]', 'leisure[47]')",0.061302,0.312567,0.066604,0.067945
"('leisure[236]', 'worker[1713]')",0.640933,0.899069,0.652077,0.683280
"('leisure[345]', 'student[197]')",0.356456,0.304301,0.334539,0.340941
"('leisure[346]', 'leisure[349]')",0.403948,0.964700,0.287290,0.320465
"('leisure[391]', 'leisure[435]')",0.428489,0.947903,0.300213,0.174184
"('leisure[435]', 'leisure[79]')",0.397596,0.552087,0.440454,0.522108
"('leisure[435]', 'worker[1645]')",0.001555,0.288343,0.120529,0.120170
